In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install tensorboard
!pip install -qqq accelerate==0.28.0
!pip install -qqq transformers==4.39.2
!pip install -qqq datasets # huggingface's lib.

In [3]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import ViTForImageClassification, ViTFeatureExtractor
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
dataset = load_dataset("/Users/aarononosala/Documents/Makerere/Classification_maize")
print(dataset)

Resolving data files:   0%|          | 0/100000 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/7445 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 100000
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 7445
    })
})


In [5]:
import os
import random

# Function to set the seed for reproducibility
def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)  # if you are using multi-GPU.
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    # The below two lines are for deterministic algorithm behavior in CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed()

In [ ]:
# Initialize the feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Preprocess the dataset
def preprocess_images(examples):
    images = [feature_extractor(image.convert("RGB")) for image in examples["image"]]
 
    examples['pixel_values'] = [image['pixel_values'][0] for image in images]
    return examples

dataset = dataset.map(preprocess_images, batched=True) #batch
dataset.set_format(type='torch', columns=['image', 'pixel_values', 'label'])

# Split the dataset into training and validation sets using Hugging Face's built-in method
train_test_split = dataset["train"].train_test_split(test_size=0.2)

# Assign the split datasets
train_dataset = train_test_split["train"] 
val_dataset = train_test_split["test"]

/Users/aarononosala/anaconda3/envs/tensorflow/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]